In [ ]:
#imports
import optuna
import sys
sys.path.append('/home/harry/personal/uni/project/individual/harry/model')

from model import *

feature_names = [
    'is_summer', 'is_autumn', 'is_winter', 'is_spring',
    'is_monday', 'is_tuesday', 'is_wednesday', 'is_thursday', 'is_friday', 'is_saturday', 'is_sunday',
    'is_weekday', 'is_weekend',
    'is_jan', 'is_feb', 'is_mar', 'is_apr', 'is_may', 'is_jun', 'is_jul', 'is_aug', 'is_sep', 'is_oct', 'is_nov', 'is_dec',
    'min_30_min_demand',
    'avg_30_min_demand',
    'max_30_min_demand',
    'avg_temp',
    'max_temp',
    'min_temp',
    'hd_next_24h',
    'cd_next_24h'
]

## Optuna Function
def objective(trial):
    # Suggest hyperparameters
    # Restrict d_model and nhead to best from first optuna to get better insights (no inf)
    d_model = 64
    nhead = 4
    

    params = {
        'learning_rate': trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
        'batch_size': trial.suggest_int("batch_size", 8, 64),
        'seq_length': trial.suggest_int("seq_length", 3, 21),
        'transformer_encoder_layer_params': {
            'd_model': d_model,
            'nhead': nhead,
            'dim_feedforward': trial.suggest_int("dim_feedforward", 128, 512),
            'dropout': trial.suggest_float("dropout", 0.0, 0.1),
            'activation': 'relu'
        },
        'transformer_layer_params': {
            'num_layers': trial.suggest_int("num_layers", 1, 5),
        },
        'dataset': '2010-2019',
        'train_test_split': '80:20',
        'features': ['sum_30_min_demand'] + feature_names,
        'visualise': False,
    }

    return train_model(params)

In [2]:
study = optuna.create_study(storage="sqlite:///db.sqlite3", study_name="2010_2019+all_features", load_if_exists=True)
study.optimize(objective, n_trials=100, n_jobs=1)

study.best_params

[I 2025-09-29 11:19:38,320] Using an existing study with name '2010_2019+all_features' instead of creating a new one.


Using device: cuda


[I 2025-09-29 11:20:08,000] Trial 3 finished with value: 3.1824056399172322 and parameters: {'learning_rate': 0.004805942134174947, 'batch_size': 47, 'seq_length': 10, 'dim_feedforward': 323, 'dropout': 0.07539051040836629, 'num_layers': 4}. Best is trial 1 with value: 3.0486290549032313.


Using device: cuda


[I 2025-09-29 11:20:32,570] Trial 4 finished with value: 3.001530727837248 and parameters: {'learning_rate': 0.0002878752531644364, 'batch_size': 56, 'seq_length': 6, 'dim_feedforward': 232, 'dropout': 0.07307726163621338, 'num_layers': 5}. Best is trial 4 with value: 3.001530727837248.


...

Using device: cuda


[I 2025-09-29 11:47:27,454] Trial 101 finished with value: 2.9500937674742924 and parameters: {'learning_rate': 0.00029515456810600256, 'batch_size': 27, 'seq_length': 3, 'dim_feedforward': 436, 'dropout': 0.05644014426158325, 'num_layers': 1}. Best is trial 99 with value: 2.7369358069832876.


Using device: cuda


[I 2025-09-29 11:47:42,227] Trial 102 finished with value: 2.9225745837947796 and parameters: {'learning_rate': 0.00024820814576021464, 'batch_size': 37, 'seq_length': 5, 'dim_feedforward': 399, 'dropout': 0.06524949770853845, 'num_layers': 1}. Best is trial 99 with value: 2.7369358069832876.


{'learning_rate': 0.00024261380969184664,
 'batch_size': 22,
 'seq_length': 3,
 'dim_feedforward': 420,
 'dropout': 0.05278116820216414,
 'num_layers': 1}